In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
train = pd.read_csv("Data/ailsa/Ailsadata2.csv", index_col=0)
test = pd.read_csv("Data/ailsa/Ailsatest2.csv")

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
train_cols = train.columns.to_list()
test_cols = test.columns.to_list()

In [37]:
train.head()

,TransactionID,isFraud,TransactionAmt,ProductCD,card3,card4,card6,addr2,dist1,P_emaildomain,...,M1,M2,M3,M4,M5,M6,M7,M8,M9,hour
1,2987000,0,68.5,W,150,discover,credit,87,19.000000,google,...,1,1,1,M2,0,1,0,0,1,0
2,2987001,0,29.0,W,150,mastercard,credit,87,118.502948,google,...,1,1,1,M0,1,1,0,0,1,0
3,2987002,0,59.0,W,150,visa,debit,87,287.000000,Microsoft,...,1,1,1,M0,0,0,0,0,0,0
4,2987003,0,50.0,W,150,mastercard,debit,87,118.502948,yahoo,...,1,1,1,M0,1,0,0,0,1,0
5,2987004,0,50.0,H,150,mastercard,credit,87,118.502948,google,...,1,1,1,M0,0,0,0,0,1,0


In [19]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, bootstrap=False)


In [44]:
y = train.isFraud

num_cols = test.select_dtypes(exclude=object).columns.to_list()

X = train[num_cols]

forest.fit(X,y)

RandomForestClassifier(bootstrap=False)

In [56]:
X_test = test[num_cols]

X_test.head()

,TransactionID,TransactionAmt,dist1,C1,C2,C3,C4,C5,C6,C7,...,D15,M1,M2,M3,M5,M6,M7,M8,M9,hour
0,3663549,31.95,1.0,6.0,6.0,0.0,0.0,3.0,4.0,0.0,...,409.0,1,1,0,0,0,1,1,1,0
1,3663550,49.00,4.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,...,634.0,1,0,0,0,0,0,0,1,0
2,3663551,171.00,2635.0,2.0,2.0,0.0,0.0,0.0,5.0,0.0,...,97.0,1,1,0,0,0,0,0,0,0
3,3663552,284.95,17.0,5.0,2.0,0.0,0.0,1.0,1.0,0.0,...,242.0,1,1,1,0,1,0,0,1,0
4,3663553,67.95,6.0,6.0,6.0,0.0,0.0,2.0,5.0,0.0,...,22.0,1,1,1,0,0,0,1,1,0


In [58]:
X_test_fill = X_test.fillna(X_test.mean())

X_test_fill.head()

,TransactionID,TransactionAmt,dist1,C1,C2,C3,C4,C5,C6,C7,...,D15,M1,M2,M3,M5,M6,M7,M8,M9,hour
0,3663549,31.95,1.0,6.0,6.0,0.0,0.0,3.0,4.0,0.0,...,409.0,1,1,0,0,0,1,1,1,0
1,3663550,49.00,4.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,...,634.0,1,0,0,0,0,0,0,1,0
2,3663551,171.00,2635.0,2.0,2.0,0.0,0.0,0.0,5.0,0.0,...,97.0,1,1,0,0,0,0,0,0,0
3,3663552,284.95,17.0,5.0,2.0,0.0,0.0,1.0,1.0,0.0,...,242.0,1,1,1,0,1,0,0,1,0
4,3663553,67.95,6.0,6.0,6.0,0.0,0.0,2.0,5.0,0.0,...,22.0,1,1,1,0,0,0,1,1,0


In [59]:
predictions = forest.predict(X_test_fill)

In [60]:
output = pd.DataFrame({'TransactionID': test.TransactionID,
                       'isFraud': predictions})

output.to_csv("Data/ailsa/testing.csv", index=False)

actually ok wtf

now try with categorical columns

In [82]:
new_train = train.drop('isFraud', axis=1)

new_train = pd.get_dummies(new_train)

In [89]:
new_test = test.copy()

new_test[num_cols]= X_test_fill

cat_cols = new_test.select_dtypes(include=object).columns.to_list()

new_test[cat_cols].fillna(new_test[cat_cols].mode())

new_test = pd.get_dummies(new_test)

,TransactionID,TransactionAmt,dist1,C1,C2,C3,C4,C5,C6,C7,...,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_verizon.net,P_emaildomain_windstream.net,P_emaildomain_yahoo,P_emaildomain_ymail.com,M4_M0,M4_M1,M4_M2
1,2987000,68.50,19.000000,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2987001,29.00,118.502948,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2987002,59.00,287.000000,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,2987003,50.00,118.502948,2,5,0,0,0,4,0,...,0,0,0,0,0,1,0,1,0,0
5,2987004,50.00,118.502948,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590536,3577535,49.00,48.000000,2,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
590537,3577536,39.50,118.502948,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
590538,3577537,30.95,118.502948,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
590539,3577538,117.00,3.000000,1,1,0,0,0,3,0,...,0,0,0,0,0,0,0,1,0,0


In [92]:
new_train = new_train.drop('card6_debit or credit', axis=1) # needed so both test and train have smae cols

In [93]:
forest_2 = RandomForestClassifier()

forest_2.fit(new_train, y)

RandomForestClassifier()

In [94]:
new_preds = forest_2.predict(new_test)

In [95]:
output = pd.DataFrame({'TransactionID': test.TransactionID,
                       'isFraud': new_preds})

output.to_csv("Data/ailsa/testing2.csv", index=False)